##INSTRUCTION:


1.   This program is trained and tested on the GPU environment. Plese make sure that your colab notebook is connected to GPU Runtime.
2.  The main aim of the below network is to observe the major differences between training from scratch and Transfer Learning (using Pre-trained weights).




Student Deatils:

In [2]:
name = 'Sachin Singh'
id = '1141124'
print(f"My name is : {name},\n My student id : {id}\nMy last digit of ID belongs to : {id[-1]}\nDataset to be selected: Scene-15 dataset and Caltech256")
print(f"My second last digit of ID belongs to : {id[-2]}\nDCNN to be selected: DenseNet ")

Importing All Important Libraries.

In [3]:
from keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import he_normal
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.layers import Input, Dense, Flatten, BatchNormalization, Dropout

Loading data, Converting images into RGB, Scalling, Batching

In [4]:
def generator(dir, gen = image.ImageDataGenerator(rescale = 1./255, validation_split=0.2), shuffle = True,
            batch_size = 1, target_size=(24, 24), class_mode = 'categorical', subset = None):
    return gen.flow_from_directory(dir,batch_size=batch_size, shuffle=shuffle, color_mode='rgb',
                                   class_mode=class_mode, target_size=target_size, subset=subset)

Defining Batch Size and Target Size

In [5]:
BS = 32
TS = (224, 224)

**CALTECH256** 

Loading Data into Train and Validate batch.

> total no of classes = 256

> total no of images = 29,744 (after cleaning)

Assignment 4 requirements 

> Randomly selecting approx 30 images per class for training.

> Remaining images are used for testing.

In [6]:
train_batch = generator('../input/caltech256-without-any-noise/256_ObjectCategories', shuffle=True, batch_size = BS, target_size = TS, subset='training')
valid_batch = generator('../input/caltech256-without-any-noise/256_ObjectCategories', shuffle=True, batch_size = BS, target_size = TS, subset='validation')

Defining Input Shape

In [7]:
from tensorflow.keras.initializers import he_normal
initial = he_normal(seed=32)

In [8]:
input_shape_densenet = (224, 224, 3)

Creating Model object with Weights None. (Training from Scratch).

> When weights None. (Training from Scratch).

> When weights = 'imagnet' (pre-trained weights, Transfer learning)

In [9]:
# comment if using pretarined
densenet_model = DenseNet169(
    include_top=False,
    weights= None,
    input_tensor=None,
    input_shape=input_shape_densenet,
    pooling=None
)

In [43]:
# uncomment for pretarined weights.
# densenet_model = DenseNet169(
#     include_top=False,
#     weights= "imagenet",
#     input_tensor=None,
#     input_shape=input_shape_densenet,
#     pooling=None
# )

In [10]:
# comment if using pretarined
densenet_model.trainable = True

In [44]:
# uncomment for transfer Learning.
# densenet_model.trainable = True
# for layer in densenet_model.layers:
#     if 'conv5' in layer.name:
#         layer.trainable = True
#     else:
#         layer.trainable = False

In [11]:
input_s = Input(input_shape_densenet)
new_layer = densenet_model(input_s)

Flatining the layer and adding FC layers.

In [12]:
new_layer = Flatten()(new_layer)

new_layer = BatchNormalization()(new_layer)

new_layer = Dense(units=768, activation='relu', kernel_initializer= initial)(new_layer)

In [13]:
new_layer = Dropout(0.4)(new_layer)

new_layer = BatchNormalization()(new_layer)

new_layer = Dense(units=512, activation='relu', kernel_initializer= initial)(new_layer)

Output layer with Softmax Activation funaction.

In [14]:
new_layer = Dropout(0.4)(new_layer)

output = Dense(units=256, activation='softmax', kernel_initializer= initial)(new_layer)

Printing Model Summary.

In [15]:
densenet_model = Model(inputs=input_s, outputs=output)
densenet_model.summary()

Compiling the model.

In [16]:
densenet_model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

Fitting the model.

This is  pretrained network for 3 Run with 10 epochs.

In [51]:
# # uncomment for transfer Learning.
# previous = densenet_model.fit(train_batch, validation_data=valid_batch, epochs=10, verbose=1)

Fitting the model.

This is model from scratch network for single Run with 50 epochs.

In [ ]:
from_scratch = densenet_model.fit(train_batch, validation_data=valid_batch, epochs=50, verbose=1)